# Exercise 2 - NumPy Linear Algebra

A very common usage of NumPy is to perform common linear algebra algorithms.  These are usually implemented efficiently for the CPU using a library like OpenBLAS.  We will see in a later tutorial how to move these computations to the GPU.

In this tutorial, we're going to look at the [Singular Value Decomposition]() of a matrix.  The matrix will represent a grayscale image, and we will see how the image can be approximated by the top N singular values and vectors.  This could be used as part of an image compression system, for example.

We will need OpenCV to read the image format, and matplotlib to display the image and our reconstruction of it.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

Next let's download an image of Bryce's dog:

In [ ]:
import urllib.request
urllib.request.urlretrieve(
    'https://drive.usercontent.google.com/download?id=1ClKrHt4-SIHaeBJdF0K3MG64jyVnt62L&export=download',
    'loonie.jpg')

Next we can read the image in grayscale mode:

In [ ]:
image = cv2.imread('loonie.jpg', cv2.IMREAD_GRAYSCALE)
image.nbytes, image.shape, image

Here we can see the image is 1600x1200 pixels, and each pixel is an unsigned 8-bit value (0-255).  Let's plot it with matplotlib to verify it looks correct:

In [ ]:
plt.imshow(image, cmap='gray')
plt.title("Bryce's Dog")
plt.show()

Yes, we can confirm that is a dog (and a very cute one at that).  Now let's start doing some linear algebra!

NumPy provides an [implementation of SVD](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html).  By selecting `full_matrices=False`, we get the singular value matrix, `S`, as a 1D vector rather than a 2D diagonal matrix.

In [ ]:
U, S, Vt = np.linalg.svd(image, full_matrices=False)
U.shape, S.shape, Vt.shape

Since the image is not square and we've not selected `full_matrices`, NumPy returns `U` as a non-square matrix, `S` as the 1D vector which is the smaller of the two dimensions, and the `Vt` matrix is a square matrix.

The signular values are returned in descending order, which we can see if we look at the first 10 elements of `S`:

In [ ]:
S[:10]

In fact, if we look at the size of the singular values, we see that the first few contribute a lot to the matrix, and then fall off very rapidly:

In [ ]:
plt.semilogy(S)

That suggests we can get a pretty good approximation of the original image with a relatively small number of terms.  We can reconstruct the image matrix by slicing the `U`, `S`, and `Vt` matrices and remultiplying them.  We will need to convert `S` back into a 2D matrix for the multiplication as well.  Note that we are using the `@` operator to perform matrix multiplication, because `*` does element-wise multiplication.

In [ ]:
# First 3 terms
nterms = 3
reconstructed = U[:, :nterms] @ np.diag(S[:nterms]) @ Vt[:nterms, :]
plt.imshow(reconstructed, cmap='gray')
plt.title('n = 3')
plt.show()

That's still pretty fuzzy, so let's check out the image with more terms included:

In [ ]:
plt.figure(figsize=(16, 4))

start, end, step = 10, 50, 10
for i in range(start, end, step):
    plt.subplot(1, (end - start) // step + 1, (i - start) // step + 1)
    reconstructed = U[:, :i] @ np.diag(S[:i]) @ Vt[:i, :]
    plt.imshow(reconstructed, cmap='gray')
    plt.title(f'n = {i}')

plt.tight_layout()
plt.show()

**Exercise: Print the compression ratio for the values of `n` used above.  This is the number of bytes of the reduced arrays added together and divided by the size of the original grayscale image array.  How does this compare to the size of the original color JPEG, which is 756473 bytes?  (Hint: `.nbytes` works on sliced arrays!)**

In [ ]:
for i in range(start, end, step):
    compress_ratio = 1  # replace with your calculation
    print(f'n = {i}: compression = {compress_ratio:.1%}')

**Exercise: Compute and display the difference in the reconstruction for n = 10 and the original image.  Use `cmap="coolwarm"` when you display the difference image.**

In [ ]:
delta = ...
plt.imshow(delta, cmap='coolwarm')